<h1> Data Handling </h1>


<h3> Importing Libraries </h3>

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import re
import requests
from datetime import datetime, timedelta


<h3> Loading the .csv files to Dataframes </h3>

In [2]:
df_arrivals = pd.read_csv("flight_arrivals_JFK_raw_data.csv")
df_departures = pd.read_csv("flight_departures_JFK_raw_data.csv")


<h3> Merging the dataframes to one dataframe called 'merged_df' </h3>

In [3]:
merged_df = pd.concat([df_arrivals, df_departures])
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS
0,05 May 2023,New York (JFK),Teterboro (TEB),GLF4 (N455MB),0:31,14:45,15:33,16:10,Landed 16:04
1,05 May 2023,Bermuda (BDA),New York (JFK),GLF4 (N455MB),2:01,12:30,13:23,13:24,Landed 14:24
2,03 May 2023,White Plains (HPN),Bermuda (BDA),GLF4 (N455MB),1:44,07:30,08:24,10:17,Landed 11:08
3,03 May 2023,Teterboro (TEB),White Plains (HPN),GLF4 (N455MB),0:22,06:00,06:33,06:24,Landed 06:55
4,30 Apr 2023,Van Nuys (VNY),Teterboro (TEB),GLF4 (N455MB),4:48,14:45,14:45,22:34,Landed 22:34
...,...,...,...,...,...,...,...,...,...
63215,29 Jan 2023,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,Landed 06:35
63216,22 Jan 2023,Mexico City (MEX),New York (JFK),B762 (XA-EFR),—,08:15,18:37,14:00,Diverted to ORD
63217,19 Jan 2023,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,Landed 18:08
63218,19 Jan 2023,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,Landed 14:34


<h4> Removing duplicates </h4>

In [68]:
merged_df = merged_df.dropna()
merged_df

<h4> Saving the file as csv </h4>

In [5]:
merged_df.to_csv("check.csv")

<h4> Verifying that there are no NULL cells that need to be handled </h4>

In [84]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183360 entries, 0 to 183359
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         183360 non-null  datetime64[ns]
 1   FROM         183360 non-null  object        
 2   TO           183360 non-null  object        
 3   AIRCRAFT     183360 non-null  object        
 4   FLIGHT TIME  183360 non-null  object        
 5   STD          183360 non-null  object        
 6   ATD          183360 non-null  object        
 7   STA          183360 non-null  object        
 8   STATUS       183360 non-null  object        
 9   DIFFERENCE   183067 non-null  float64       
 10  AGE          182083 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 15.4+ MB


<h4> In the below command, we are looking for a non-null cells that contain '-' and delete all those rows </h4>

In [9]:
mask = merged_df.apply(lambda x: x.str.contains('—')).any(axis=1)
merged_df = merged_df[~mask]
print(merged_df)

              DATE                  FROM                    TO       AIRCRAFT  \
0      05 May 2023        New York (JFK)       Teterboro (TEB)  GLF4 (N455MB)   
1      05 May 2023         Bermuda (BDA)        New York (JFK)  GLF4 (N455MB)   
2      03 May 2023    White Plains (HPN)         Bermuda (BDA)  GLF4 (N455MB)   
3      03 May 2023       Teterboro (TEB)    White Plains (HPN)  GLF4 (N455MB)   
4      30 Apr 2023        Van Nuys (VNY)       Teterboro (TEB)  GLF4 (N455MB)   
...            ...                   ...                   ...            ...   
63214  30 Jan 2023        New York (JFK)         Chicago (ORD)  A306 (XA-UYR)   
63215  29 Jan 2023     Mexico City (MEX)        New York (JFK)  A306 (XA-UYR)   
63217  19 Jan 2023        New York (JFK)         Chicago (ORD)  B762 (XA-LRC)   
63218  19 Jan 2023  Guatemala City (GUA)        New York (JFK)  B762 (XA-LRC)   
63219  19 Jan 2023     Mexico City (MEX)  Guatemala City (GUA)  B762 (XA-LRC)   

      FLIGHT TIME    STD   

In [12]:
merged_df.to_csv("check.csv")

In [73]:
merged_df = pd.read_csv("check.csv")

<h4> This line of code is resetting the indexes </h4>

In [74]:
merged_df = merged_df.reset_index(drop=True)

<h4> Deleting 'Landed' part from the STATUS column </h4>


In [75]:
merged_df['STATUS'] = merged_df['STATUS'].str.replace('Landed ', '')
merged_df

,Unnamed: 0.1,Unnamed: 0,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE
0,0,0,05 May 2023,New York (JFK),Teterboro (TEB),GLF4 (N455MB),0:31,14:45,15:33,16:10,16:04,-6.0
1,1,1,05 May 2023,Bermuda (BDA),New York (JFK),GLF4 (N455MB),2:01,12:30,13:23,13:24,14:24,60.0
2,2,2,03 May 2023,White Plains (HPN),Bermuda (BDA),GLF4 (N455MB),1:44,07:30,08:24,10:17,11:08,51.0
3,3,3,03 May 2023,Teterboro (TEB),White Plains (HPN),GLF4 (N455MB),0:22,06:00,06:33,06:24,06:55,31.0
4,4,4,30 Apr 2023,Van Nuys (VNY),Teterboro (TEB),GLF4 (N455MB),4:48,14:45,14:45,22:34,22:34,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
183355,183355,183355,30 Jan 2023,New York (JFK),Chicago (ORD),A306 (XA-UYR),2:13,06:30,09:16,10:28,10:29,1.0
183356,183356,183356,29 Jan 2023,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,06:35,995.0
183357,183357,183357,19 Jan 2023,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,18:08,3.0
183358,183358,183358,19 Jan 2023,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,14:34,-51.0


<h4> Deleteing the two first columns of indexes 'Unnamed' because they are not necessary </h4>


In [76]:
merged_df = merged_df.drop('Unnamed: 0', axis=1)
merged_df = merged_df.drop('Unnamed: 0.1', axis=1)
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE
0,05 May 2023,New York (JFK),Teterboro (TEB),GLF4 (N455MB),0:31,14:45,15:33,16:10,16:04,-6.0
1,05 May 2023,Bermuda (BDA),New York (JFK),GLF4 (N455MB),2:01,12:30,13:23,13:24,14:24,60.0
2,03 May 2023,White Plains (HPN),Bermuda (BDA),GLF4 (N455MB),1:44,07:30,08:24,10:17,11:08,51.0
3,03 May 2023,Teterboro (TEB),White Plains (HPN),GLF4 (N455MB),0:22,06:00,06:33,06:24,06:55,31.0
4,30 Apr 2023,Van Nuys (VNY),Teterboro (TEB),GLF4 (N455MB),4:48,14:45,14:45,22:34,22:34,0.0
...,...,...,...,...,...,...,...,...,...,...
183355,30 Jan 2023,New York (JFK),Chicago (ORD),A306 (XA-UYR),2:13,06:30,09:16,10:28,10:29,1.0
183356,29 Jan 2023,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,06:35,995.0
183357,19 Jan 2023,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,18:08,3.0
183358,19 Jan 2023,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,14:34,-51.0


<h4> In the below chunk of code, we are running in a loop on all the cells of 'STATUS' column and we are inserting to a new column called 'DITTERENCE' the difference bewteen 'STA' (scheduled time of arrival) and 'STATUS' (actual time of arrival) for each line accordingly  </h4>

In [17]:
difflist = []
for i in range(len(merged_df['STATUS'])):
    if ':' in merged_df['STA'][i] and ':' in merged_df['STATUS'][i]:
        time1 = datetime.strptime(merged_df['STA'][i], '%H:%M')
        time2 = datetime.strptime(merged_df['STATUS'][i], '%H:%M')
        diff = time2-time1
        total_minutes = diff.seconds // 60
        hours, minutes = divmod(total_minutes, 60)
        if diff.days < 0 and hours >= 23:
            diff = timedelta(1) - diff
            total_minutes = diff.seconds // 60
            total_minutes = -total_minutes
        else:
            total_minutes = diff.seconds // 60
        difflist.append(total_minutes)
    else:
            difflist.append(None)



merged_df['DIFFERENCE'] = difflist
merged_df

,Unnamed: 0,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE
0,0,05 May 2023,New York (JFK),Teterboro (TEB),GLF4 (N455MB),0:31,14:45,15:33,16:10,16:04,-6.0
1,1,05 May 2023,Bermuda (BDA),New York (JFK),GLF4 (N455MB),2:01,12:30,13:23,13:24,14:24,60.0
2,2,03 May 2023,White Plains (HPN),Bermuda (BDA),GLF4 (N455MB),1:44,07:30,08:24,10:17,11:08,51.0
3,3,03 May 2023,Teterboro (TEB),White Plains (HPN),GLF4 (N455MB),0:22,06:00,06:33,06:24,06:55,31.0
4,4,30 Apr 2023,Van Nuys (VNY),Teterboro (TEB),GLF4 (N455MB),4:48,14:45,14:45,22:34,22:34,0.0
...,...,...,...,...,...,...,...,...,...,...,...
183355,183355,30 Jan 2023,New York (JFK),Chicago (ORD),A306 (XA-UYR),2:13,06:30,09:16,10:28,10:29,1.0
183356,183356,29 Jan 2023,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,06:35,995.0
183357,183357,19 Jan 2023,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,18:08,3.0
183358,183358,19 Jan 2023,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,14:34,-51.0


In [18]:
merged_df.to_csv("check.csv")

<h4> Verifying that there are no NULL cells that need to be handled </h4>

In [21]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183067 entries, 0 to 183359
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   183067 non-null  int64  
 1   DATE         183067 non-null  object 
 2   FROM         183067 non-null  object 
 3   TO           183067 non-null  object 
 4   AIRCRAFT     183067 non-null  object 
 5   FLIGHT TIME  183067 non-null  object 
 6   STD          183067 non-null  object 
 7   ATD          183067 non-null  object 
 8   STA          183067 non-null  object 
 9   STATUS       183067 non-null  object 
 10  DIFFERENCE   183067 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 16.8+ MB


In [20]:
merged_df.dropna(subset=['DIFFERENCE'], inplace=True)

<h4> In this step we are creating a new list called 'arcrft_reg' which represent the actual aircraft of the flight and we are copying into this list only the string that inside of the brackets </h4>

In [19]:
arcrft_reg = [item.split('(')[-1].strip(')') for item in merged_df['AIRCRAFT']]
arcrft_reg




['N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',
 'N455MB',

<h4> Saving to a DataFrame </h4>

In [21]:
df_arcrft_reg = pd.DataFrame(arcrft_reg)
df_arcrft_reg

,0
0,N455MB
1,N455MB
2,N455MB
3,N455MB
4,N455MB
...,...
183062,XA-UYR
183063,XA-UYR
183064,XA-LRC
183065,XA-LRC


<h4> Removing duplicates </h4>

In [22]:
df_arcrft_reg = df_arcrft_reg.drop_duplicates()
df_arcrft_reg

,0
0,N455MB
132,GLF4
139,B-222J
140,B-2077
141,B-223A
...,...
182896,OE-IFB
183019,XA-LRC
183023,XA-GGL
183031,XA-EFR


<h5> Saving to a csv file </h5>

In [63]:
df_arcrft_reg.to_csv("aircraft_reg_raw.csv")

<h4> after we save the file in the line above, we worked on it in the Arrivals scraping Notebook. in the below line of code we are reading from the csv to a dataframe and we cleaning all the none-null cells than containg "-" in them, because they are meaningless </h4>

In [4]:
df_arcrft_age = pd.read_csv("aircraft_reg_new.csv")

In [6]:
df_arcrft_age = df_arcrft_age.replace("-", np.nan)

In [7]:
df_arcrft_age = df_arcrft_age.dropna()

In [8]:
df_arcrft_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4876 entries, 0 to 4879
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4876 non-null   int64 
 1   FLIGHT REG  4876 non-null   object
 2   AGE         4876 non-null   object
dtypes: int64(1), object(2)
memory usage: 152.4+ KB


<h4> In the below step we are looking for Brand New airplanes and changing them to 0, in addition, we are deleting from each cell of 'AGE' and delete the 'years' from the string </h4>

In [32]:
df_arcrft_age.replace('Brand new', 0, inplace=True)
df_arcrft_age['AGE'] = df_arcrft_age['AGE'].apply(lambda x: str(x).replace(' year', '') if isinstance(x, str) else x)


<h1> LOOKS AWSOME! </h1>

In [79]:
df_arcrft_age

,Unnamed: 0,FLIGHT REG,AGE
0,0,N455MB,19
1,1,B-222J,0
2,2,B-2077,13
3,3,B-223A,0
4,4,B-222N,0
...,...,...,...
4875,4875,OE-IFB,19
4876,4876,XA-LRC,36
4877,4877,XA-GGL,31
4878,4878,XA-EFR,35


<h2> In the following 3 chunks of code we are doing the follwing: <br> 1. creating a new column called 'AGE'. <br> 2. extracting the string of the aircraft inside the brackets. <br> 3. for each row, we are looking in the _df_arcrft_age for the age and copying it to the 'AGE' column in the merged DataFrame </h2>

In [80]:
merged_df['AGE'] = pd.Series([])
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE
0,2023-05-05,New York (JFK),Teterboro (TEB),GLF4 (N455MB),0:31,14:45,15:33,16:10,16:04,-6.0,NaN
1,2023-05-05,Bermuda (BDA),New York (JFK),GLF4 (N455MB),2:01,12:30,13:23,13:24,14:24,60.0,NaN
2,2023-05-03,White Plains (HPN),Bermuda (BDA),GLF4 (N455MB),1:44,07:30,08:24,10:17,11:08,51.0,NaN
3,2023-05-03,Teterboro (TEB),White Plains (HPN),GLF4 (N455MB),0:22,06:00,06:33,06:24,06:55,31.0,NaN
4,2023-04-30,Van Nuys (VNY),Teterboro (TEB),GLF4 (N455MB),4:48,14:45,14:45,22:34,22:34,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
183355,2023-01-30,New York (JFK),Chicago (ORD),A306 (XA-UYR),2:13,06:30,09:16,10:28,10:29,1.0,NaN
183356,2023-01-29,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,06:35,995.0,NaN
183357,2023-01-19,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,18:08,3.0,NaN
183358,2023-01-19,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,14:34,-51.0,NaN


In [81]:
merged_df['AIRCRAFT'] = [item.split('(')[-1].strip(')') for item in merged_df['AIRCRAFT']]
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE
0,2023-05-05,New York (JFK),Teterboro (TEB),N455MB,0:31,14:45,15:33,16:10,16:04,-6.0,NaN
1,2023-05-05,Bermuda (BDA),New York (JFK),N455MB,2:01,12:30,13:23,13:24,14:24,60.0,NaN
2,2023-05-03,White Plains (HPN),Bermuda (BDA),N455MB,1:44,07:30,08:24,10:17,11:08,51.0,NaN
3,2023-05-03,Teterboro (TEB),White Plains (HPN),N455MB,0:22,06:00,06:33,06:24,06:55,31.0,NaN
4,2023-04-30,Van Nuys (VNY),Teterboro (TEB),N455MB,4:48,14:45,14:45,22:34,22:34,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
183355,2023-01-30,New York (JFK),Chicago (ORD),XA-UYR,2:13,06:30,09:16,10:28,10:29,1.0,NaN
183356,2023-01-29,Mexico City (MEX),New York (JFK),XA-UYR,3:51,08:15,01:44,14:00,06:35,995.0,NaN
183357,2023-01-19,New York (JFK),Chicago (ORD),XA-LRC,2:04,16:55,17:04,18:05,18:08,3.0,NaN
183358,2023-01-19,Guatemala City (GUA),New York (JFK),XA-LRC,4:13,10:30,09:21,15:25,14:34,-51.0,NaN


In [82]:
for index, row in merged_df.iterrows():
    aircraft = row['AIRCRAFT']
    df_filtered = df_arcrft_age.loc[df_arcrft_age['FLIGHT REG'] == aircraft, 'AGE']
    if not df_filtered.empty:
        age = df_filtered.iloc[0]
        merged_df.at[index, 'AGE'] = age
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE
0,2023-05-05,New York (JFK),Teterboro (TEB),N455MB,0:31,14:45,15:33,16:10,16:04,-6.0,19
1,2023-05-05,Bermuda (BDA),New York (JFK),N455MB,2:01,12:30,13:23,13:24,14:24,60.0,19
2,2023-05-03,White Plains (HPN),Bermuda (BDA),N455MB,1:44,07:30,08:24,10:17,11:08,51.0,19
3,2023-05-03,Teterboro (TEB),White Plains (HPN),N455MB,0:22,06:00,06:33,06:24,06:55,31.0,19
4,2023-04-30,Van Nuys (VNY),Teterboro (TEB),N455MB,4:48,14:45,14:45,22:34,22:34,0.0,19
...,...,...,...,...,...,...,...,...,...,...,...
183355,2023-01-30,New York (JFK),Chicago (ORD),XA-UYR,2:13,06:30,09:16,10:28,10:29,1.0,30
183356,2023-01-29,Mexico City (MEX),New York (JFK),XA-UYR,3:51,08:15,01:44,14:00,06:35,995.0,30
183357,2023-01-19,New York (JFK),Chicago (ORD),XA-LRC,2:04,16:55,17:04,18:05,18:08,3.0,36
183358,2023-01-19,Guatemala City (GUA),New York (JFK),XA-LRC,4:13,10:30,09:21,15:25,14:34,-51.0,36


<h4> checking for null cells that need to be clean </h4>

In [45]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183067 entries, 0 to 183359
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   183067 non-null  int64         
 1   DATE         183067 non-null  datetime64[ns]
 2   FROM         183067 non-null  object        
 3   TO           183067 non-null  object        
 4   AIRCRAFT     183067 non-null  object        
 5   FLIGHT TIME  183067 non-null  object        
 6   STD          183067 non-null  object        
 7   ATD          183067 non-null  object        
 8   STA          183067 non-null  object        
 9   STATUS       183067 non-null  object        
 10  DIFFERENCE   183067 non-null  float64       
 11  AGE          181793 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 22.2+ MB


In [85]:
merged_df.dropna(subset=['AGE'], inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182083 entries, 0 to 183359
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         182083 non-null  datetime64[ns]
 1   FROM         182083 non-null  object        
 2   TO           182083 non-null  object        
 3   AIRCRAFT     182083 non-null  object        
 4   FLIGHT TIME  182083 non-null  object        
 5   STD          182083 non-null  object        
 6   ATD          182083 non-null  object        
 7   STA          182083 non-null  object        
 8   STATUS       182083 non-null  object        
 9   DIFFERENCE   181793 non-null  float64       
 10  AGE          182083 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 16.7+ MB


In [86]:
merged_df.dropna(subset=['DIFFERENCE'], inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181793 entries, 0 to 183359
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         181793 non-null  datetime64[ns]
 1   FROM         181793 non-null  object        
 2   TO           181793 non-null  object        
 3   AIRCRAFT     181793 non-null  object        
 4   FLIGHT TIME  181793 non-null  object        
 5   STD          181793 non-null  object        
 6   ATD          181793 non-null  object        
 7   STA          181793 non-null  object        
 8   STATUS       181793 non-null  object        
 9   DIFFERENCE   181793 non-null  float64       
 10  AGE          181793 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 16.6+ MB


In [88]:
merged_df['DAY'] = merged_df['DATE'].dt.day
merged_df['MONTH'] = merged_df['DATE'].dt.month
merged_df['YEAR'] = merged_df['DATE'].dt.year
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE,DAY,MONTH,YEAR
0,2023-05-05,New York (JFK),Teterboro (TEB),N455MB,0:31,14:45,15:33,16:10,16:04,-6.0,19,5,5,2023
1,2023-05-05,Bermuda (BDA),New York (JFK),N455MB,2:01,12:30,13:23,13:24,14:24,60.0,19,5,5,2023
2,2023-05-03,White Plains (HPN),Bermuda (BDA),N455MB,1:44,07:30,08:24,10:17,11:08,51.0,19,3,5,2023
3,2023-05-03,Teterboro (TEB),White Plains (HPN),N455MB,0:22,06:00,06:33,06:24,06:55,31.0,19,3,5,2023
4,2023-04-30,Van Nuys (VNY),Teterboro (TEB),N455MB,4:48,14:45,14:45,22:34,22:34,0.0,19,30,4,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183355,2023-01-30,New York (JFK),Chicago (ORD),XA-UYR,2:13,06:30,09:16,10:28,10:29,1.0,30,30,1,2023
183356,2023-01-29,Mexico City (MEX),New York (JFK),XA-UYR,3:51,08:15,01:44,14:00,06:35,995.0,30,29,1,2023
183357,2023-01-19,New York (JFK),Chicago (ORD),XA-LRC,2:04,16:55,17:04,18:05,18:08,3.0,36,19,1,2023
183358,2023-01-19,Guatemala City (GUA),New York (JFK),XA-LRC,4:13,10:30,09:21,15:25,14:34,-51.0,36,19,1,2023


In [90]:
merged_df['DAY OF WEEK'] = merged_df['DATE'].dt.day_name()
merged_df

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE,DAY,MONTH,YEAR,DAY OF WEEK
0,2023-05-05,New York (JFK),Teterboro (TEB),N455MB,0:31,14:45,15:33,16:10,16:04,-6.0,19,5,5,2023,Friday
1,2023-05-05,Bermuda (BDA),New York (JFK),N455MB,2:01,12:30,13:23,13:24,14:24,60.0,19,5,5,2023,Friday
2,2023-05-03,White Plains (HPN),Bermuda (BDA),N455MB,1:44,07:30,08:24,10:17,11:08,51.0,19,3,5,2023,Wednesday
3,2023-05-03,Teterboro (TEB),White Plains (HPN),N455MB,0:22,06:00,06:33,06:24,06:55,31.0,19,3,5,2023,Wednesday
4,2023-04-30,Van Nuys (VNY),Teterboro (TEB),N455MB,4:48,14:45,14:45,22:34,22:34,0.0,19,30,4,2023,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183355,2023-01-30,New York (JFK),Chicago (ORD),XA-UYR,2:13,06:30,09:16,10:28,10:29,1.0,30,30,1,2023,Monday
183356,2023-01-29,Mexico City (MEX),New York (JFK),XA-UYR,3:51,08:15,01:44,14:00,06:35,995.0,30,29,1,2023,Sunday
183357,2023-01-19,New York (JFK),Chicago (ORD),XA-LRC,2:04,16:55,17:04,18:05,18:08,3.0,36,19,1,2023,Thursday
183358,2023-01-19,Guatemala City (GUA),New York (JFK),XA-LRC,4:13,10:30,09:21,15:25,14:34,-51.0,36,19,1,2023,Thursday


In [92]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181793 entries, 0 to 183359
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         181793 non-null  datetime64[ns]
 1   FROM         181793 non-null  object        
 2   TO           181793 non-null  object        
 3   AIRCRAFT     181793 non-null  object        
 4   FLIGHT TIME  181793 non-null  object        
 5   STD          181793 non-null  object        
 6   ATD          181793 non-null  object        
 7   STA          181793 non-null  object        
 8   STATUS       181793 non-null  object        
 9   DIFFERENCE   181793 non-null  float64       
 10  AGE          181793 non-null  object        
 11  DAY          181793 non-null  int64         
 12  MONTH        181793 non-null  int64         
 13  YEAR         181793 non-null  int64         
 14  DAY OF WEEK  181793 non-null  object        
dtypes: datetime64[ns](1), float64(1), 

In [96]:
merged_df = merged_df.drop('DATE', axis=1)
merged_df

,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE,AGE,DAY,MONTH,YEAR,DAY OF WEEK
0,New York (JFK),Teterboro (TEB),N455MB,0:31,14:45,15:33,16:10,16:04,-6.0,19,5,5,2023,Friday
1,Bermuda (BDA),New York (JFK),N455MB,2:01,12:30,13:23,13:24,14:24,60.0,19,5,5,2023,Friday
2,White Plains (HPN),Bermuda (BDA),N455MB,1:44,07:30,08:24,10:17,11:08,51.0,19,3,5,2023,Wednesday
3,Teterboro (TEB),White Plains (HPN),N455MB,0:22,06:00,06:33,06:24,06:55,31.0,19,3,5,2023,Wednesday
4,Van Nuys (VNY),Teterboro (TEB),N455MB,4:48,14:45,14:45,22:34,22:34,0.0,19,30,4,2023,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183355,New York (JFK),Chicago (ORD),XA-UYR,2:13,06:30,09:16,10:28,10:29,1.0,30,30,1,2023,Monday
183356,Mexico City (MEX),New York (JFK),XA-UYR,3:51,08:15,01:44,14:00,06:35,995.0,30,29,1,2023,Sunday
183357,New York (JFK),Chicago (ORD),XA-LRC,2:04,16:55,17:04,18:05,18:08,3.0,36,19,1,2023,Thursday
183358,Guatemala City (GUA),New York (JFK),XA-LRC,4:13,10:30,09:21,15:25,14:34,-51.0,36,19,1,2023,Thursday


In [97]:
new_order = ['DAY', 'MONTH', 'YEAR', 'DAY OF WEEK', 'FROM', 'TO', 'AIRCRAFT', 'AGE', 'FLIGHT TIME', 'STD', 'ATD', 'STA', 'STATUS', 'DIFFERENCE']
merged_df = merged_df.reindex(columns=new_order)
merged_df

,DAY,MONTH,YEAR,DAY OF WEEK,FROM,TO,AIRCRAFT,AGE,FLIGHT TIME,STD,ATD,STA,STATUS,DIFFERENCE
0,5,5,2023,Friday,New York (JFK),Teterboro (TEB),N455MB,19,0:31,14:45,15:33,16:10,16:04,-6.0
1,5,5,2023,Friday,Bermuda (BDA),New York (JFK),N455MB,19,2:01,12:30,13:23,13:24,14:24,60.0
2,3,5,2023,Wednesday,White Plains (HPN),Bermuda (BDA),N455MB,19,1:44,07:30,08:24,10:17,11:08,51.0
3,3,5,2023,Wednesday,Teterboro (TEB),White Plains (HPN),N455MB,19,0:22,06:00,06:33,06:24,06:55,31.0
4,30,4,2023,Sunday,Van Nuys (VNY),Teterboro (TEB),N455MB,19,4:48,14:45,14:45,22:34,22:34,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183355,30,1,2023,Monday,New York (JFK),Chicago (ORD),XA-UYR,30,2:13,06:30,09:16,10:28,10:29,1.0
183356,29,1,2023,Sunday,Mexico City (MEX),New York (JFK),XA-UYR,30,3:51,08:15,01:44,14:00,06:35,995.0
183357,19,1,2023,Thursday,New York (JFK),Chicago (ORD),XA-LRC,36,2:04,16:55,17:04,18:05,18:08,3.0
183358,19,1,2023,Thursday,Guatemala City (GUA),New York (JFK),XA-LRC,36,4:13,10:30,09:21,15:25,14:34,-51.0


In [98]:
merged_df.to_csv("Merged_final.csv")